In [2]:
import numpy as np
import json

In [132]:
bool(np.isclose(np.sum(np.eye(2)), 0))

False

In [151]:
def write_group(f, members, name):
    # tiling code needs to be updated for more than 2D
    result = {'name': name, 'size': len(members), 'members': []}
    
    has_identity = False
    for m in members:
        # skip extra identities
        if np.allclose(np.eye(m.shape[0]), m):
            if not has_identity:
                has_identity = True
            else:
                continue
        if not has_identity:
            raise ValueError('Must start with identity')
        i = np.linalg.inv(m)        
        # tiling meaning inner Ndim x Ndim is diag only and last column is non-zero
        diag = np.isclose(np.sum(m[:-1, :-1]**2), np.trace(m[:-1, :-1]**2))
        tiling = bool(not np.isclose(np.sum(m[:-1,-1]**2), 0) and diag)
        result['members'].append({'g': list(m.flatten()), 'i': list(i.flatten()), 't': tiling})
    # sort them - identity first and tiling last    
    result['members'].sort(
        key = lambda m: int(m['t']))
    json.dump(result, f, indent=True)
    print('Wrote group with', len(members), 'members')
    
def cgroup_2d(n):
    thetas = np.linspace(0, 2 * np.pi, n + 1)[:-1]
    r = []
    for t in thetas:
        s, c = np.sin(t), np.cos(t)
        r.append(np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]]))
    return r
def sgroup_2d(n):
    thetas = np.arange(0, n) / n
    r = [np.eye(3)]
    for t in thetas:
        s, c = np.sin(2 * np.pi * t), np.cos(2 * np.pi * t)
        r.append(np.array([[c, s, 0], [s, -c, 0], [0, 0, 1]]))
    return r

def trans_2d(theta):
    v = [np.eye(3) for _ in range(9)]
    for i in range(3):
        for j in range(3):            
            v[i * 3 + j][0, 2] = (i - 1) * np.cos(theta)
            v[i * 3 + j][1, 2] = (j - 1) * np.sin(theta)
    # put identity at beginning
    del v[1 * 3 + 1]
    v.insert(0, np.eye(3))
    return v

def outer(g1, g2, remove_dups = True):
    r = []
    for i in g1:
        for j in g2:            
            add = True
            if remove_dups:                
                for ri in r:
                    if np.allclose(i @ j, ri):
                        add = False
                        break            
            if add:
                r.append(i @ j)
    return r

In [171]:
with open('group.json', 'w') as f:
    #g = outer(cgroup_2d(3), trans_2d(np.radians(45)))
    g = trans_2d(np.radians(45))
    write_group(f, g, 'my_group')


Wrote group with 9 members


In [178]:
Ntot = 64
Ns = Ntot // len(g)
N = int(np.floor(np.sqrt(Ns)))
Ns = N * N
Ntot = Ns * len(g)
box = 3
with open('start_positions.xyz', 'w') as f:
    p = np.linspace(0, box, N)
    for i in range(N):
        for j in range(N):
            f.write(f'{p[i] - box / 2} {p[j] - box / 2}\n')
    for i in range(Ntot - N):
        f.write('0.0 0.0\n')
print(Ns, Ntot, Ntot / len(g))

4 36 4.0


In [17]:
(N ** 2 // 7) * 7

4095

In [169]:
len(g)

27